In [1]:
import pandas as pd
from collections import Counter
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\results.csv")
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [3]:
# get the fifa world cup games only
#dfFIFA = df.loc[df.tournament == 'FIFA World Cup'].reset_index()
dfFIFA = df

In [4]:
# get the year column
dfFIFA['Year'] = dfFIFA['date'].apply(lambda x: int(str.split(x,"-")[0]))

In [6]:
#dfFIFA = dfFIFA.drop(columns = ['index','date','city','country','neutral'])
dfFIFA = dfFIFA.drop(columns = ['date','city','neutral'])
dfFIFA['home_games'] = 1
dfFIFA['away_games'] = 1
dfFIFA.head()

,home_team,away_team,home_score,away_score,tournament,country,Year,home_games,away_games
0,Scotland,England,0,0,Friendly,Scotland,1872,1,1
1,England,Scotland,4,2,Friendly,England,1873,1,1
2,Scotland,England,2,1,Friendly,Scotland,1874,1,1
3,England,Scotland,2,2,Friendly,England,1875,1,1
4,Scotland,England,3,0,Friendly,Scotland,1876,1,1


In [7]:
# get the year and country finals held
dfFIFACountry = dfFIFA[['Year','country']]
dfFIFACountry = dfFIFACountry.drop_duplicates().reset_index().drop(columns = ['index'])
dfFIFACountry

,Year,country
0,1872,Scotland
1,1873,England
2,1874,Scotland
3,1875,England
4,1876,Scotland
...,...,...
9421,2021,Senegal
9422,2021,Ivory Coast
9423,2021,Madagascar
9424,2021,Nigeria


In [8]:
#dfFIFA.loc[(dfFIFA.home_team=='Chile') & (dfFIFA.Year == 1930)]

In [9]:
dfFIFA_home = dfFIFA.groupby(['Year','home_team']).sum().rename(columns = {'away_score':'home_goals_conceded'}).drop(columns ='away_games') #.reset_index().set_index('Year')
dfFIFA_home.head()

,,home_score,home_goals_conceded,home_games
Year,home_team,,,
1872,Scotland,0,0,1
1873,England,4,2,1
1874,Scotland,2,1,1
1875,England,2,2,1
1876,Scotland,7,0,2


In [10]:
dfFIFA_away = dfFIFA.groupby(['Year','away_team']).sum().rename(columns = {'home_score':'away_goals_conceded'}).drop(columns = ['home_games'])#.reset_index().set_index('Year')
dfFIFA_away.head()

,,away_goals_conceded,away_score,away_games
Year,away_team,,,
1872,England,0,0,1
1873,Scotland,4,2,1
1874,England,2,1,1
1875,Scotland,2,2,1
1876,England,3,0,1


In [11]:
dfFIFA_away_ri = dfFIFA_away.reset_index()
dfFIFA_away_ri.loc[dfFIFA_away_ri.Year == 1930]

,Year,away_team,away_goals_conceded,away_score,away_games
617,1930,Argentina,4,2,1
618,1930,Aruba,2,1,1
619,1930,Austria,4,3,2
620,1930,Basque Country,0,1,1
621,1930,Belgium,5,10,3
622,1930,Brazil,0,4,1
623,1930,Bulgaria,12,2,2
624,1930,Chile,3,1,1
625,1930,China PR,3,3,1
626,1930,Czechoslovakia,7,7,5


In [14]:
dfFIFAdata = pd.concat([dfFIFA_away, dfFIFA_home.reindex(dfFIFA_away.index)], axis=1).reset_index().set_index('Year').rename(columns = {'away_team':'team'})
dfFIFAdata = dfFIFAdata.fillna(0)
dfFIFAdata

,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games
Year,,,,,,,
1872,England,0,0,1,0.0,0.0,0.0
1873,Scotland,4,2,1,0.0,0.0,0.0
1874,England,2,1,1,0.0,0.0,0.0
1875,Scotland,2,2,1,0.0,0.0,0.0
1876,England,3,0,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2021,United States,1,2,1,11.0,1.0,2.0
2021,Uzbekistan,0,2,1,0.0,1.0,1.0
2021,Wales,3,1,1,2.0,0.0,2.0


In [15]:
dfFIFA_total_avg_1 = dfFIFA.groupby(['Year']).mean().rename(columns = {'home_score':'hs_mean','away_score':'as_mean'}).drop(columns = ['home_games','away_games'])
dfFIFA_total_avg_1.head()

,hs_mean,as_mean
Year,,
1872,0.0,0.0
1873,4.0,2.0
1874,2.0,1.0
1875,2.0,2.0
1876,3.5,0.0


In [16]:
dfFIFA_total_avg_2 = dfFIFA_total_avg_1.copy().rename(columns = {'hs_mean':'as_conceded_mean','as_mean':'hs_conceded_mean'})
dfFIFA_total_avg_2.head()

,as_conceded_mean,hs_conceded_mean
Year,,
1872,0.0,0.0
1873,4.0,2.0
1874,2.0,1.0
1875,2.0,2.0
1876,3.5,0.0


In [17]:
dfFIFA_total_avg = pd.concat([dfFIFA_total_avg_1, dfFIFA_total_avg_2], axis=1, join='inner')
dfFIFA_total_avg.head()

,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
Year,,,,
1872,0.0,0.0,0.0,0.0
1873,4.0,2.0,4.0,2.0
1874,2.0,1.0,2.0,1.0
1875,2.0,2.0,2.0,2.0
1876,3.5,0.0,3.5,0.0


In [18]:
dfFIFAdata = dfFIFAdata.merge(dfFIFA_total_avg, on='Year').reset_index()
dfFIFAdata.head()

,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean
0,1872,England,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1873,Scotland,4,2,1,0.0,0.0,0.0,4.0,2.0,4.0,2.0
2,1874,England,2,1,1,0.0,0.0,0.0,2.0,1.0,2.0,1.0
3,1875,Scotland,2,2,1,0.0,0.0,0.0,2.0,2.0,2.0,2.0
4,1876,England,3,0,1,0.0,0.0,0.0,3.5,0.0,3.5,0.0


In [19]:
dfFIFAdata['H_AttackS'] = (dfFIFAdata['home_score']/dfFIFAdata['home_games'])/dfFIFAdata['hs_mean']
dfFIFAdata['A_AttackS'] = (dfFIFAdata['away_score']/dfFIFAdata['away_games'])/dfFIFAdata['as_mean']
dfFIFAdata['H_DefenseS'] = (dfFIFAdata['home_goals_conceded']/dfFIFAdata['home_games'])/dfFIFAdata['hs_conceded_mean']
dfFIFAdata['A_DefenseS'] = (dfFIFAdata['away_goals_conceded']/dfFIFAdata['away_games'])/dfFIFAdata['as_conceded_mean']

In [20]:
dfFIFAdata = dfFIFAdata.set_index(['Year','team'])
dfFIFAdata.head()

,,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,as_mean,as_conceded_mean,hs_conceded_mean,H_AttackS,A_AttackS,H_DefenseS,A_DefenseS
Year,team,,,,,,,,,,,,,,
1872,England,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1873,Scotland,4,2,1,0.0,0.0,0.0,4.0,2.0,4.0,2.0,NaN,1.0,NaN,1.000000
1874,England,2,1,1,0.0,0.0,0.0,2.0,1.0,2.0,1.0,NaN,1.0,NaN,1.000000
1875,Scotland,2,2,1,0.0,0.0,0.0,2.0,2.0,2.0,2.0,NaN,1.0,NaN,1.000000
1876,England,3,0,1,0.0,0.0,0.0,3.5,0.0,3.5,0.0,NaN,NaN,NaN,0.857143


In [22]:
# this dataset contains statistic yearly wise. Earlier i calculated for a particular tournament
dfFIFAdata.to_csv(r"C:\Users\Suwani Gunasekara\OneDrive\Documents\Sem 7 ML\Mid term\data_statistics_yearly.csv")